## Наследование. Множественное наследование. MRO. Композиция. Делегирование. Псевдозакрытые атрибуты классов

### Множественное наследование и MRO

Множественное наследование (подмешивание классов) - когда класс-ребенок имеет больше одного класса-родителя. В таком случае наследуются все методы и атрибуты суперклассов. 

In [96]:
class Employee:
    '''Общий класс-родитель (дедушка)'''
  def __init__(self, name, surname):
    self.name = name
    self.surname = surname
    self.__salary = 350  # _Employee__salary
    self.bankaccount = 0

  def work(self, *args):
    raise NotImplementedError

class Linguist(Employee):
    '''Класс-мама'''
  def __init__(self, name, surname):
    Employee.__init__(self, name, surname)
    self.__salary = self._Employee__salary * 1.5
    self.publications = []
    
  def work(self, hours, name_of_publ):
    print('Working...')
    sleep(hours)
    self.bankaccount += self.salary * hours
    self.publications.append(name_of_publ)
    print(f'{self.name} {self.surname} has published a paper {name_of_publ} in Voprosy Yazykoznaniya')

  def readlingpapers(self, lingpaper):
    print('Reading...')
    sleep(len(lingpaper) // 10)
    print(f'{self.name} {self.surname} has read {lingpaper}')

class Programmer(Employee):
    '''Класс-папа'''
  def __init__(self, name, surname):
    Employee.__init__(self, name, surname)
    self.__salary *= self._Employee__salary * 2.5
    self.projects = []

  def work(self, hours, name_of_project):
    print('Working...')
    sleep(hours)
    self.bankaccount += self.salary * hours
    self.projects.append(name_of_project)
    print(f'{self.name} {self.surname} has committed a repo {name_of_project} to GitHub')

  def learnpython(self, hours):
    print('Studying...')
    sleep(hours)
    print(f'{self.name} {self.surname} has learned a bit of Python')

In [82]:
class ComputerLinguist(Linguist, Programmer):
    '''Наследуемся одновременно от лингвиста и от программиста'''
  def __init__(self, name, surname):
    Linguist.__init__(self, name, surname)  # явным образом вызываем init класса "лингвист"
    Programmer.__init__(self, name, surname)
    self.salary *= 0.9

  def work(self, hours, name_of_paper_with_code):
    print('Working...')
    sleep(hours)
    self.publications.append(name_of_paper_with_code)
    self.projects.append(name_of_paper_with_code)
    self.bankaccount += self.salary * hours
    print(f'{self.name} {self.surname} has published {name_of_paper_with_code} in ACL Papers')

При таком раскладе, если мы вдруг не определили бы у класса-ребенка метод work, а у обоих классов-родителей он был, питон пошел бы искать у того класса, который мы в скобках написали первым. 

![Image](https://media.geeksforgeeks.org/wp-content/uploads/220px-diamond_inheritance-svg.png)

Эта вещь и называется Method Resolution Order = MRO. Когда мы вызываем какой-нибудь метод класса, питон попросту ищет такой метод в пространстве имен сперва самого этого класса, потом, если не находит, то у класса-родителя, от которого наследуем первым, а если у него нет, то у его родителя, и так далее. 

Что касается динамических атрибутов, то если есть одноименные, то все решает порядок вызовов init: кого последним вызвали, того и зарплата. 

### Композиция и делегирование

Другой способ сочетать классы при проектировании программы - композиция. Это когда мы вкладываем экземпляры классов в другие классы и вызываем их методы внутри определения класса. 

In [65]:
class Employee:
    '''Класс общего назначения'''
  def __init__(self, name, surname):
    self.name = name
    self.surname = surname
    self.salary = 350
    self.bankaccount = 0

  def work(self, *args):
    raise NotImplementedError

  def sleep(self, message):
    print(message)

class Linguist:
  def __init__(self, name, surname, salary_coef):
    self.employee = Employee(name, surname)  # создаем экземпляр класса Employee и складываем его в атрибут класса Лингвист. 
    self.employee.salary *= salary_coef  # у самого лингвиста при этом нет атрибутов, кроме employee, поэтому если нам нужна зарплата, обращаемся напрямую к атрибутам employee
  
  def work(self, hours):
    print('Working...')
    sleep(hours)
    self.employee.bankaccount += hours * self.employee.salary

  def sleep(self, message):
    self.employee.sleep(message)  # делегирование

  def __getattr__(self, attr):
    '''А чтобы можно было обращаться к атрибутам вложенного класса, приходится делегировать'''
    print(f'{attr} is called')
    return getattr(self.employee, attr) # встроенная функция 

  def __setattr__(self, attr, value):
    print(f'trying to set attr {attr}')
    self.__dict__[attr] = value

Магические методы \_\_getattr\_\_ и \_\_setattr\_\_ вызываются, когда мы обращаемся к неопределенному (не существующему) атрибуту через точку. То есть, когда они вызываются в этом конкретном случае: у класса Linguist есть только один атрибут - employee. Если мы к нему обратимся, эти методы не сработают. Но если мы обратимся к атрибуту name, которого у Лингвиста нет, потому что он есть только у вложенного экземпляра класса Employee, то вызовется метод \_\_getattr\_\_, а тут-то мы и перенаправим на атрибут вложенного экземпляра. 

Это и называется делегирование. В примере выше оно происходит в методах sleep, getattr & setattr.

Почему нам приходится таким странным образом переопределять setattr? Потому что если мы напишем просто 

    def __setattr__(self, attr, value):
        self.attr = value
        
Возникнет рекурсия: точка опять вызовет этот же метод. Кстати говоря, зациклить можно и getattr, если переопределить его для получения атрибутов этого же класса. 

Поэтому мы идем обходным путем и используем встроенный атрибут класса \_\_dict\_\_. Что это за атрибуты такие? Это специальные зарезервированные атрибуты, некоторые из них по умолчанию у любого класса появляются, некоторые обычно появляются, но могут нет (\_\_dict\_\_ как раз может не быть). Этот самый \_\_dict\_\_ - это действительно всего лишь обычнейший словарь, где в качестве ключей хранятся имена атрибутов, а в значениях, собственно, их значения. 

У класса есть еще парочка полезных атрибутов, например, \_\_class\_\_ возвращает ссылку из экземпляра на класс. В этой ссылке содержится имя класса \_\_name\_\_ и последовательность \_\_bases\_\_, в которой лежат классы-родители. 

In [73]:
class Employee:
  def __init__(self, name, surname):
    self.name = name
    self.surname = surname
    self.salary = 350
    self.bankaccount = 0

  def work(self, *args):
    raise NotImplementedError

  def __repr__(self):
    return f"{self.__class__.__name__}('{self.name}', '{self.surname}')"  # так с помощью этих атрибутов можно определить метод repr, чтобы меньше хардкодить

In [ ]:
class ComputerLinguist(Linguist, Programmer):
    '''Наследуемся одновременно от лингвиста и от программиста'''
  def __init__(self, name, surname):
    for base in self.__class__.__bases__:
        base.__init__(self, name, surname)
    # Альтернатива закомментированному коду
    # Linguist.__init__(self, name, surname)  # явным образом вызываем init класса "лингвист"
    # Programmer.__init__(self, name, surname)
    self.salary *= 0.9

Композиция выглядит более громоздкой и сложной, чем наследование, но у нее есть свои области применения. Обычно композиция + делегирование используется, когда нам нужен какой-нибудь класс-контролер, который сам по себе ничего не делает, но, например, следит за состоянием другого класса. Можно логировать (выводить логи про каждый пшик), например, если в переопределенных методах getattr & setattr прописать какие-нибудь принты, можно отслеживать обращения к атрибутам. 

Композиция также позволяет агрегировать экземпляры другого класса:

In [76]:
class ARD:
    '''отдел Advanced Research & Development '''
  def __init__(self, *args):
    self.members = list(args)  # куча лингвистов

  def add_member(self, linguist):
    self.members.append(linguist)

  def raise_salary(self, coef):
    '''массово всему отделу повышаем зарплатку'''
    for linguist in self.members:
      linguist.salary *= coef

  def workEveryone(self, hour):
    '''или гоним работать лентяев'''
    for linguist in self.members:
      linguist.work(hour)

Фишку с делегированием и переопределением getattr & setattr также ограниченно можно использовать для реализации инкапсуляции: закрыть доступ к атрибутам класса. 

Ну и вот пример класса-контроллера, который сам по себе ничего не делает, только следит за работой внутреннего класса:

In [80]:
class Wrapper:
  def __init__(self, object):
    self.wrapped = object
  
  def __getattr__(self, attr):
    print(f'Big brother is watching you, here is your {attr}')
    return getattr(self.wrapped, attr)

In [86]:
w = Wrapper(gorbunova)

In [87]:
w.name

Big brother is watching you, here is your name


'Ira'

Похожим образом работают дескрипторы классов и декораторы. 

### Псевдозакрытые атрибуты классов

Напомню, в ООП есть такая концепция, как инкапсуляция: она о том, что мы должны иметь доступ к классу только через его интерфейс, а его внутренности должны быть от нас скрыты. Например, в объектно-ориентированном языке Java это реализуется с помощью операторов public & private:

    public class tokenisation {
        public static void main(String[] args) throws Exception
        {
            System.setProperty("file.encoding", "UTF-8");
            Scanner scan = new Scanner(System.in, "UTF-8");
            final DecimalFormat df = new DecimalFormat("0.00");

            System.out.println("Enter file path:");
            String path = scan.nextLine();
            File file = new File(path);

            ArrayList<String> text = new ArrayList<String>();

            Pattern pattern = Pattern.compile("[a-zА-Яа-яёЁ]+(-[a-zА-Яа-яёЁ]+)*", Pattern.CASE_INSENSITIVE);

            BufferedReader br = new BufferedReader(new FileReader(file));
            String str;
            while ((str = br.readLine()) != null) {
                Matcher matcher = pattern.matcher(str);
                while (matcher.find()) {
                    String match = matcher.group();
                    text.add(match);
                }

            }
            br.close();
            scan.close();

            Set<String> unique = new HashSet<String>(text);

            float lexdiv = (float)unique.size() / (float)text.size() * 100;

            System.out.println(df.format(lexdiv));

        }
    }
    
Так выглядит код на этом языке. Можно заметить, что здесь встречается словечко public, которое сообщает, что этот класс и эту функцию можно вызывать из других мест. Если бы было private, то мы не имели бы доступа к этому объекту извне. 

У питона таких готовых инструментов нет, но можно закрывать атрибуты и имитировать инкапсуляцию разными способами. В частности, в питоне есть псевдозакрытые атрибуты: это скорее про конвенции, чем про синтаксис, но тем не менее. 

Если мы хотим сделать какой-то атрибут класса закрытым, то мы должны назвать его с двойным нижним подчеркиванием перед именем:

    self.__salary
    
Тогда внутри класса мы можем к нему обращаться по этому имени, а вне тоже вообще-то можем, но тогда придется писать \_Linguist\_\_salary, потому что такие атрибуты интерпретатор автоматически переименовывает при определении класса. 

#### Свойства (properties)

Мы с вами пользовались для делегирования переопределением методов \_\_setattr\_\_ & \_\_getattr\_\_. Эти методы вызываются только тогда, когда мы обращаемся к атрибуту экземпляра, которого у него нет. Например:

In [3]:
class A:
    def __init__(self, a):
        self.a = a
    def __getattr__(self, attr):
        print('getattr')

In [4]:
a = A(1)
a.a # обращаемся к атрибуту, который есть

1

In [5]:
a.b  # обращаемся к атрибуту, которого нет

getattr


Нужно быть осторожнее с этими методами, потому что если их неаккуратно переопределить, можно уйти в бесконечный цикл: если мы напишем в getattr getattr(self, attr), уйдем в рекурсию, можете проверить. 

Еще осторожнее нужно быть с методами \_\_getattribute\_\_ & \_\_setattribute\_\_, которые делают то же самое, но для всех атрибутов вообще, существуют они или нет. 

Если же нам нужно переопределить способ обращения к какому-то атрибуту класса (например, добавить проверку при присваивании), можно воспользоваться встроенной функцией property, которая обеспечивает интерфейс для атрибутов экземпляра класса. Сам метод принимает четыре аргумента:

    property(get, set, del, doc)
    
Как правило, если вы используете этот метод, вы точно захотите передать ему get, а дальше по убыванию: doc передается реже всего. Ни один из них по сути не обязательный, но get, наверное, хотелось бы передавать. 

Как это все реализуется:

In [71]:
class Person:
    def __init__(self):
        self.__name = ''
        
    def getname(self):
        print('getting name')
        return self.__name
        
    def setname(self, name):
        if name.isalpha():
            self.__name = name
    
    def delname(self):
        print('deleting name')
        del self.__name
    name = property(getname, setname, delname)  # property(get, set, del, doc)

In [72]:
p = Person()

In [73]:
p.name = 'Vasya'

In [74]:
p.name

getting name


'Vasya'

В каком порядке я определяю функции getname, setname & delname (а также как я их называю - можно называть их как угодно), неважно: важно только, в каком порядке я их передаю в метод property. 

Переменная name находится на верхнем уровне определения класса, на одном уровне с определениями методов. Но при этом сам атрибут name (точнее говоря, \_\_name) будет принадлежать конкретному экземпляру. Мы самому классу приписали свойство, которое теперь будет взаимодействовать со всеми его экземплярами, устанавливая им внутренний атрибут \_\_name. Это надо четко себе уяснить: name - не атрибут экземпляра. Это атрибут класса, который хранит в себе *дескриптор*. А дескриптор - это объект, который управляет внутренним атрибутом \_\_name. 

Да:  property - это разновидность дескриптора вообще. Давайте посмотрим про дескрипторы. 

#### Дескрипторы. Введение

Дескриптор - это такой объект питона, который имплементирует метод протокола дескриптора. Что такое протокол дескриптора? (descriptor protocol) Это то, каким образом питон работает с атрибутами объектов (классов или экземпляров). Мы с атрибутами что можем делать:

- Запрашивать их значение (\_\_get\_\_)
- Устанавливать их значение (\_\_set\_\_)
- Удалять их (\_\_delete\_\_)
- и устанавливать им имя и объект-хозяина (\_\_set\_name\_\_)

Все эти вещи и переопределяет дескриптор, чтобы можно было их делать каким-нибудь особенным образом. Дескрипторы бывают двух видов: data & non-data. Первый переопределяет метод \_\_set\_\_, а второй нет. 

Итак, как это все выглядит:

In [114]:
class Temperature:
    """descriptor"""
    def __get__(self, person, type=None):
        if hasattr(person, 'temp'):
            return person.temp
        raise AttributeError
    
    def __set__(self, person, value):
        if 30 <= value <= 42:
            person.temp = value
        else:
            print('Can\'t set!')

class Person:
    temperature = Temperature()

In [115]:
vasya = Person()

In [116]:
vasya.temperature = 43

Can't set!


In [117]:
vasya.temperature = 36

Дескриптор - это **отдельный класс**. Это и имеется в виду, когда мы говорим "объект". Когда мы определяем класс Person, у него заводим его статический атрибут, в который помещаем конкретный экземпляр класса "дескриптор". Теперь, когда мы будем писать vasya.temperature, мы на самом деле обращаемся к дескриптору, а не к атрибуту с целочисленным значением! А дескриптор уже вызывает нужные методы. То есть, что происходит, если совсем подробно:

    vasya = Person()  
    # у васи, как у экземпляра класса, образуется атрибут Temperature(), 
    то есть, создается конкретный объект-дескриптор. Никаких динамических атрибутов у васи нет!
    
    vasya.temperature 
    # мы обращаемся к дескриптору! У него в этот момент неявным образом вызывается метод __get__, который принимает что? 
    Сам экземпляр дескриптора (self), объект person = vasya, type, он же owner - это класс (мы можем его не передавать). 
    
    vasya.temperature = 36
    # мы опять обращаемся к дескриптору, но оператор присваивания заставляет вызваться метод __set__, который принимает опять
    сам экземпляр дескриптора, объект vasya и значение 36. Именно поэтому пишем person.temp = value: мы записываем значение уже в реальный динамический атрибут. 
    
    del vasya.temperature 
    # должен был бы вызвать метод __delete__, но мы его не определили. 

Дескрипторы, как и более простой их вариант property, обычно используются с синтаксисом декораторов (скоро посмотрим). Нужны они все для того же самого: для проверок, логов и всяческого разнообразного контроля над атрибутами, а еще чтоб жизнь малиной не казалась. :)

#### Статические методы и методы класса

Статические методы не принимают экземпляр класса: собственно говоря, эти методы могли бы существовать как отдельные функции, но часто их удобно запихивать в класс, чтобы а) не импортировать из модуля кучу имен б) лучшая структура программы в) переопределять их при наследовании!

Методы класса вместо экземпляра принимают сам класс. 

Допустим, у нас есть класс "Пицца", у которого хотим сделать метод, чтобы вычислять тупо площадь пиццы по ее диаметру: при этом начинка пиццы, которая нам важна для других методов, здесь не играет роли. Можно это сделать отдельной функцией:

In [10]:
import math

def pizza_area(d):
    return math.pi * (d / 2) ** 2

class Pizza:
    def __init__(self, *ingrs):
        self.ingredients = ingrs
    
    def price(self):
        return len(self.ingredients)  # это, конечно, глупость, но мне лень писать адекватно :)

В таком варианте, если мы захотим импортировать класс "Пицца" в другой скрипт, придется импортировать и функцию вместе с ним. Хочется включить метод в класс:

In [11]:
class Pizza:
    def __init__(self, *ingrs):
        self.ingredients = ingrs
    
    def price(self):
        return len(self.ingredients)  
    
    def pizza_area(d):
        return math.pi * (d / 2) ** 2

Такой код не будет работать:

In [8]:
pizza = Pizza('cheese', 'sausage', 'olives')
pizza.pizza_area(30)

TypeError: pizza_area() takes 1 positional argument but 2 were given

Будет работать такой:

In [12]:
Pizza.pizza_area(30)

706.8583470577034

Но, кстати говоря, во 2 питоне и такой не сработает (сейчас никто не пользуется 2 питоном, но мало ли...)

Тут-то и можно использовать статические методы:

In [13]:
class Pizza:
    def __init__(self, *ingrs):
        self.ingredients = ingrs
    
    def price(self):
        return len(self.ingredients)  
    
    def pizza_area(d):
        return math.pi * (d / 2) ** 2
    pizza_area = staticmethod(pizza_area)

In [14]:
pizza = Pizza('cheese', 'sausage', 'olives')
pizza.pizza_area(30)

706.8583470577034

Ура, все заработало и так. Теперь мы можем как хотим обращаться к нашему статическому методу, и возможно, с точки зрения логики будет правильно обращаться к нему все равно от экземпляра. 

Метод класса работает примерно так же, только еще принимает аргумент - ссылочку на сам класс. Можно устроить этакое:

In [15]:
class Pizza:
    def __init__(self, *ingrs):
        self.ingredients = ingrs
    
    def price(self):
        return len(self.ingredients)  
    
    def pizza_area(cls, d):
        return f'{cls.__name__}: {math.pi * (d / 2) ** 2}'
    pizza_area = classmethod(pizza_area)

In [16]:
pizza = Pizza('cheese', 'sausage', 'olives')
pizza.pizza_area(30)

'Pizza: 706.8583470577034'